# Lecture 8

Please inlcude your names below
Also, please edit the name of the file and include the names of the two(or three) people answering

Pair answering the assignment: Tim Moser, David Steiger, Dominik Frauchiger  
Pair giving feedback: Eleonora Pura, Christian Skorski

## Selenium Download
For the next exercises you will have to download selenium. 

You can read more about the webdriver here (https://chromedriver.chromium.org), but if you want to go straight to the download, go to https://chromedriver.storage.googleapis.com/index.html?path=80.0.3987.106/ and download your version. 

Moreover, in your terminal type `pip install selenium`. 

Once this is done, you should be able to run:
- `from selenium import webdriver`
- `browser = webdriver.Chrome([the path where you put the googlechromedriver])`

In case of any issues, the https://chromedriver.chromium.org website has some straightforward info on common bugs. 


In [2]:
from selenium import webdriver
browser = webdriver.Chrome("chromedriver.exe")
browser.quit()

### 1. Rate limiting

1. By now, you are familiar with 3 APIs, namely Google Books, NYT, and Dribble. For each one, find and copy the rules about rate limits. Next, pick one and try to exceed the rate limit; explain what you do and what reaction you get from the API.

Google Books: 100'000 per day. [Source](https://stackoverflow.com/questions/35302157/google-books-api-rate-limiting-information/35302667).
    
NYT: 10 per minute and 4'000 per day [Source](https://developer.nytimes.com/faq).

Dribbble: 60 per min and 10'000 per day. [Source](https://speca.io/speca/dribbble-api).

2. In the next problem you will check how many requests you can send to Google Search before getting blocked. Websites protect themselves from automated crawling by checking requests that come from the same computer in a small time frame and after a while, they won't respond to the request. A valid response would be "Response 200", which you can see if you just print the response of `requests.get('https://www.google.com/search?q=zurich')`. 

The question is
a) how many requests does it take to get blocked (when you first get a response other than 200)?
b) What is the number of a blocked response and what does it exactly stand for (Google response XXX). If you still can:) 

In [10]:
import requests
from IPython.display import clear_output
res = requests.get('https://www.google.com/search?q=zurich')

print("Before\n", res.status_code)

count = 1
while res.status_code == 200:
    count += 1
    res = requests.get('https://www.google.com/search?q=zurich')
    print ("Try ", count, "Res: ", res.status_code)
    clear_output(wait=True)

print("Requests: ", count)
print("After\n", res.status_code)

Requests:  697
After
 429


**Answer:** After ~697 tries we got the status code 429 'Too Many Requests'.

### 2. Selenium sessions

Go to a website of your choice where you have an account. It can for example be the New York Times APi website where you created a login last time but also tutti.ch, comparis, whatever simple website you often use.

Using Selenium create a session where you 
1. go to the main website 
2. log in 
3. click on an element of your choice 
4. scroll to the bottom of the page
5. then save the page. 

When logging in, you will have to find the name of the login form and submit your credentials to it and then click the login button. Here you find an example for a login using selenium but in case you decide to use this help, Facebook should not be your chosen website. https://crossbrowsertesting.com/blog/test-automation/automate-login-with-selenium/

In [4]:
import time

browser = webdriver.Chrome("chromedriver.exe")

# you have to first "register" an account here (will be deleted automatically after some time).
# use as username dodo and as password pass.

browser.get("https://sopra-fs20-dfrauc-client.herokuapp.com")

username = browser.find_element_by_xpath("/html/body/div/div/div[2]/div/div/div/input[1]")
username.send_keys("dodo")

password = browser.find_element_by_xpath("/html/body/div/div/div[2]/div/div/div/input[2]")
password.send_keys("pass")

login_btn = browser.find_element_by_xpath("/html/body/div/div/div[2]/div/div/div/div/button")
login_btn.click()

# no scroll since page not long enough

time.sleep(1)

first_user = browser.find_element_by_xpath("/html/body/div/div/div[2]/div/div/div/ul/li[1]/a")
first_user.click()

with open('thePage.html', 'w') as f:
    f.write(browser.page_source)

browser.quit()

### 3. Measuring personalization

In this exercise you will have to imitate the study described in class on a website of your interest. You will have to measure differences in the content that you receive back from the website under varying treatments. 

You will have to choose a website and a treatment. Use selenium for this exercise as well. 
- As for websites, you can pick an online store, or traveling site, some news site, Google News.. basically try to pick something that you suspect gives different results for different searchers. 
- Examples for treatments would be location, being logged in with an account, history with the website, being on a phone vs a desktop, etc. 
- You can try to pick multiple searches to make sure you are measuring real phenomenon, not only noise
- You can include a control treatment in case you suspect there's A/B testing or noise in how the pages look
- Finally you have to pick a measure for the differences on the page. In case you receive items on a page, for example URLs or products, you can define an overlap metric. In case the page is more unstructured, come up with an explanation for how you define differences.

As your answer, explain which of the above you chose, how you implemented the experiment, and what difference you found in the pages you collected. 

You can find more infor on how to run multiple browsers at the same time here: https://crossbrowsertesting.com/blog/selenium/run-test-multiple-browsers-parallel-selenium/

In [30]:
# We chose ebay
# We suspect that the page shows different products depending on localation

site = "https://www.ebay.com/sch/i.html?_nkw=shirt"

# Soundcloud Switzerland

ch_browser = webdriver.Chrome("chromedriver.exe")

ch_browser.get(site)
time.sleep(1)
ch_title = ch_browser.find_element_by_xpath("/html/body/div[5]/div[5]/div[2]/div[1]/div[2]/ul/li[1]/div/div[2]/a/h3")

print("Schweizer Seite: ", ch_title.text)

# Soundcloud USA

proxy = "64.225.24.13:3128"
desired_capabilities = webdriver.DesiredCapabilities.CHROME.copy()
desired_capabilities['proxy'] = {
    "httpProxy": proxy,
    "ftpProxy": proxy,
    "sslProxy": proxy,
    "noProxy": None,
    "proxyType": "MANUAL",
    "class": "org.openqa.selenium.Proxy",
    "autodetect": False
}

us_browser = webdriver.Chrome("chromedriver.exe", desired_capabilities=desired_capabilities)

us_browser.get(site)
time.sleep(1)
us_title = us_browser.find_element_by_xpath("/html/body/div[4]/div[4]/div[2]/div[1]/div[2]/ul/li[1]/div/div[2]/a/h3")

print("US Seite: ", us_title.text)

print("Are the same? ", ch_title.text == us_title.text)

ch_browser.quit()
us_browser.quit()

Schweizer Seite:  Nike T Shirt Mens Gym Sports Tee Top Size S Med Large XL XXL Black Navy Red Blue
US Seite:  The ClassiX Slimming Body Toning Shirt Original Quality Men Slimming Body Shaper
Are the same?  False


Congratulations for completing the third notebook! Now it’s time for feedback.
1.	Pass your solution to the other pair in your group.
2.	Include your feedback in the other pair’s notebook. Don’t forget to add your names at the top.
3.	Return the notebook with feedback to the original pairs.
4.	Upload your notebook, with the feedback included by the other pair on OLAT.

You can think of/suggest (among other things)
 - improvements in the code (e.g. readability, efficiency)
 - improvements in the answers (e.g. are they easy to understand, are they correct, how can they be improved?)
 - point out differences (e.g. are there any differences between the responses of the two pairs? if yes what are they, what is the cause, and in which way can they be useful?)
 
Not all suggestions about the type of feedback apply to all types of questions. Try to give feedback in a meaningful and constructive way. 

Below there is space for giving feedback. This space should be used only by the other pair in your group.

Feedback here